<a href="https://colab.research.google.com/github/Chiamakac/TRAININGS/blob/main/TRAININGS/TRAININGS/Alignment/Projection_Work/TAG_ENG/TRIAL/LEVEL_THREE_Entity_Mapping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Read in the English text
# 2. Read in the Igbo text
# 3. Read in the English tagged text

In [4]:
from collections import Counter
import pickle

**Read and explore NER tagged English text**

In [5]:
!wget -c https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/spaCy%20en_tags%20corrected%20FINAL%20USED.txt
!wget -c https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/projection_en_bitext.txt

--2022-10-13 09:51:30--  https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/spaCy%20en_tags%20corrected%20FINAL%20USED.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288470 (282K) [text/plain]
Saving to: ‘spaCy en_tags corrected FINAL USED.txt’

spaCy en_tags corre 100%[===================>] 281.71K  --.-KB/s    in 0.03s   

2022-10-13 09:51:30 (10.9 MB/s) - ‘spaCy en_tags corrected FINAL USED.txt’ saved [288470/288470]

--2022-10-13 09:51:30--  https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/projection_en_bitext.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercon

In [6]:
tagged_sents = open('/content/spaCy en_tags corrected FINAL USED.txt').readlines()
tagged_sents = [tagged.strip() for tagged in tagged_sents]

In [ ]:
len(tagged_sents)

6633

View the first 10 entries

In [ ]:
tagged_sents[:10]

['1, Nigeria, LOC|Omoyele Sowore, PER',
 '5, Omoyele Sowore, PER|more than 120 days, DATE',
 '8, Sowore, PER',
 '12, the middle of the day, DATE',
 '13, every year, DATE|the end of every year, DATE',
 '17, today, DATE',
 '18, 1984, DATE|Nigeria, LOC|Omoyele Sowore, PER',
 '19, Monaco, LOC|Nigeria, LOC',
 '31, first, ORDINAL|Omoyele Sowore, PER',
 '32, 30 million, CARDINAL|Nigerian, NORP']

**Extract and explore the Igbo text from the En-Ig Bilingual Text**

In [ ]:
def clean_text(text):
  text = text.replace(')','')
  text = text.replace('\\','')
  return text

In [ ]:
lines = open('projection_en_bitext.txt').readlines()
ig_text = []
for line in lines:
  try:
    ig_text.append(line.strip().split("\', ")[1])
  except:
    ig_text.append(line.strip().split('\", ')[1])
[clean_text(t) for t in ig_text[:10]]
# len(ig_text)

**Convert tagged entities into a dict - `tagged_entities`**

This helps with processing each of the lines one at a time

In [7]:
tagged_entities = {}
for line in tagged_sents:
  lnum, line = line.strip().split(', ', 1)
  tagged_entities[lnum] = line.split('|')
tagged_entities, len(tagged_entities)

({'1': ['Nigeria, LOC', 'Omoyele Sowore, PER'],
  '5': ['Omoyele Sowore, PER', 'more than 120 days, DATE'],
  '8': ['Sowore, PER'],
  '12': ['the middle of the day, DATE'],
  '13': ['every year, DATE', 'the end of every year, DATE'],
  '17': ['today, DATE'],
  '18': ['1984, DATE', 'Nigeria, LOC', 'Omoyele Sowore, PER'],
  '19': ['Monaco, LOC', 'Nigeria, LOC'],
  '31': ['first, ORDINAL', 'Omoyele Sowore, PER'],
  '32': ['30 million, CARDINAL', 'Nigerian, NORP'],
  '34': ['EFCC, ORG', 'more than $ 7 billion, MONEY'],
  '35': ['Wilson Uwujaren, PER',
   'Mohammed Idris, PER',
   'December 2019, DATE',
   'the Ikoyi High Court, ORG',
   'Lagos, LOC'],
  '36': ['Abia, LOC'],
  '37': ['Wilson Uwujaren, PER',
   'Mohammed Idris, PER',
   'December 2019, DATE',
   'the Ikoyi High Court, ORG',
   'Lagos, LOC'],
  '38': ['Sun, DATE', 'Slok Nigeria Ltd, ORG', 'EFCC, ORG'],
  '39': ['Abia, LOC', '12 years, DATE', 'more than 7.5 billion, MONEY'],
  '40': ['FFK, PER', 'Patience Jonathan, PER', 'Chur

**Creating the mapping dictionary for future use**

There will be two-steps:
1. Automatic: Look for the entity in the Igbo text. If found, apply the same tag to the Igbo version else indicate "Not found!"

2. Manual: Look for all the entities not found in the `mapping_dict` and manually insert them

In [ ]:
# #  '1621': ['Nigeria, LOC', 'Muhammadu Buhari, PER', 'second, ORDINAL'],
# count_translate, found_count = 0, 0
# mapping_dict = {}
# for i, tagged in tagged_entities.items():
#   for entity_tag in tagged:
#     try:
#       entity, tag = entity_tag.rsplit(', ', 1)
#       found = ig_text[int(i)].find(entity)
#       if found>=0:
#         #transfer the Igbo tags
#         # ig_text[int(i)] = ig_text[int(i)].replace(entity, f"[{entity} - {tag}]")
#         if entity.lower() not in mapping_dict:
#           mapping_dict[entity.lower()] = {
#             'ig':entity.lower(),
#             'tag':{tag}}
#           found_count+=1
#       else:
#         if entity.lower() not in mapping_dict:
#           mapping_dict[entity.lower()] = {
#             'ig': f'Not found line {i}',
#             'tag':{tag}}
#           count_translate+=1
#     except:
#         print(f"Line {i}: Could not process '{entity}':\n{ig_text[int(i)]}\n")
# print(count_translate, found_count)

In [10]:
# count the remaining entities not found
def count_not_found():
  return sum([values['ig'].startswith('Not found') for entity, values in mapping_dict.items()])

#save the current version of mapping dictionary
def save_dict(m_dict):
  with open('mapping_dict.pkl', 'wb') as handle:
    pickle.dump(m_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print("Mapping dictionary successfully saved to 'mapping_dict.pkl'")

#load the latest version of the mapping dictionary
def load_dict(fname_pkl):
  with open(fname_pkl, 'rb') as handle:
    return pickle.load(handle)

**Correcting the Mapping Dictionary**

In [ ]:
for entity, values in mapping_dict.items():
  if values['ig'].startswith('Not found'):
    index = int(values['ig'].rsplit(' ',1)[1])
    print(f"Line {index}: {entity}\n-{tagged_entities[str(index)]}\n-{ig_text[index].lower()}")
    ig_entity = input('Enter Igbo entity: ')
    if ig_entity.lower() != 'exit':
      mapping_dict[entity]['ig'] = ig_entity
      print(f"Remaining: {count_not_found()}")
    else:
      break

Line 2461: the ministry of national planning
-['the ministry of national planning, ORG']
-"mana ngalaba na-ahụ maka nhazi, - nke a na-akpọ ministry of national planning n'asụsụ bekee - kwuru na ihe niile ndị omeiwu kwesịọrị iji hazie bọjeetị dị n'ime akwụkwọ enyere ha.")
Enter Igbo entity: ministry of national planning
Remaining: 1746
Line 2462: november last year
-['Ben Akabueze, PER', 'november last year, DATE', 'Muhammadu Buhari, PER']
-"ben akabueze, bụ onyeisi ngalaba na-ahazi mkpata na mmefu kwuru na ihe niile ndi omeiwu chọrọ dị n'akwụkwọ onyeisiala muhammadu buhari nyere ndị omeiwu n'ọnwa nọvemba nke afọ gara aga.")
Enter Igbo entity: nọvemba nke afọ gara aga
Remaining: 1745
Line 2465: the ministry of finances
-['the ministry of finances, ORG', 'Nigeria, LOC']
-"ndị ọkachamara n'ihe banyere akụnaụba obodo na-ekwu na naịjirịa agaghị ahazi akụnauba obodo ọfụma ma ya bụrụ na bọjeetị a apụtaghị.")
Enter Igbo entity: exit


**Updating the mapping dictionary**

In [ ]:
update_mdict = input('Do you want to update mapping dictionary? [Y/N]')
if update_mdict.lower() == 'y':
  save_dict(mapping_dict)

Do you want to update mapping dictionary? [Y/N]y
Mapping dictionary successfully saved to 'mapping_dict.pkl'


**Loading the mapping dictionary**

In [ ]:
load_mdict = input('Do you want to load mapping dictionary? [Y/N]')
if load_mdict.lower() == 'y':
  mapping_dict = load_dict('mapping_dict.pkl')
  print("Mapping dictionary successfully loaded from 'mapping_dict.pkl'")


---

#### **Using the `mapping_dict` to transfer NER tags from Eng to Igbo**

In [1]:
# from collections import Counter
import pickle

Read the tagged English corpus

In [2]:
!wget -c https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/spaCy%20en_tags%20corrected%20FINAL%20USED.txt

--2022-10-13 09:53:08--  https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/spaCy%20en_tags%20corrected%20FINAL%20USED.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288470 (282K) [text/plain]
Saving to: ‘spaCy en_tags corrected FINAL USED.txt’

spaCy en_tags corre 100%[===================>] 281.71K  --.-KB/s    in 0.02s   

2022-10-13 09:53:08 (13.6 MB/s) - ‘spaCy en_tags corrected FINAL USED.txt’ saved [288470/288470]



In [3]:
tagged_sents = open('/content/spaCy en_tags corrected FINAL USED.txt').readlines()
tagged_sents = [tagged.strip() for tagged in tagged_sents]

In [4]:
tagged_entities = {}
for line in tagged_sents:
  lnum, line = line.strip().split(', ', 1)
  tagged_entities[int(lnum)] = line.split('|')

Read the English-Igbo BiText

In [5]:
!wget -c https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/projection_en_bitext.txt

--2022-10-13 09:53:49--  https://raw.githubusercontent.com/Chiamakac/TRAININGS/main/Alignment/Projection_Work/TAG_ENG/projection_en_bitext.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



Extract the Igbo Text only

In [6]:
ig_text = open('projection_ig_text.txt').readlines()

**Loading the mapping dictionary**

In [11]:
load_mdict = input('Do you want to load mapping dictionary? [Y/N]')
if load_mdict.lower() == 'y':
  mapping_dict = load_dict('mapping_dict.pkl')
  print("Mapping dictionary successfully loaded from 'mapping_dict.pkl'")

Do you want to load mapping dictionary? [Y/N]y
Mapping dictionary successfully loaded from 'mapping_dict.pkl'


In [14]:
mapping_dict


{'nigeria': {'ig': 'naịjirịa', 'tag': {'LOC'}},
 'omoyele sowore': {'ig': 'omoyele sowore', 'tag': {'PER'}},
 'more than 120 days': {'ig': 'karịrị abalị otu narị na iri abụọ',
  'tag': {'DATE'}},
 'sowore': {'ig': 'sowore', 'tag': {'PER'}},
 'the middle of the day': {'ig': "n'etiti ehihie", 'tag': {'DATE'}},
 'every year': {'ig': 'kwa afọ na ngwụsị afọ ọbụla', 'tag': {'DATE'}},
 'the end of every year': {'ig': 'ngwụsị afọ ọbụla', 'tag': {'DATE'}},
 'today': {'ig': 'taa', 'tag': {'DATE'}},
 '1984': {'ig': '1984', 'tag': {'DATE'}},
 'monaco': {'ig': 'monaco', 'tag': {'LOC'}},
 'first': {'ig': 'mbụ', 'tag': {'ORDINAL'}},
 '30 million': {'ig': 'nde iri atọ naịjirịa', 'tag': {'CARDINAL'}},
 'nigerian': {'ig': 'nde iri atọ naịjirịa', 'tag': {'NORP'}},
 'efcc': {'ig': 'efcc', 'tag': {'ORG'}},
 'more than $ 7 billion': {'ig': "n'ihe karịrị wayo ijeri narịa asaa",
  'tag': {'MONEY'}},
 'wilson uwujaren': {'ig': 'wilson uwujaren', 'tag': {'PER'}},
 'mohammed idris': {'ig': 'mohammed idris', 'tag

**Working with Mapping Dictionary**

You can `[V]iew`, `[A]dd`, `[R]emove` or `[U]pdate` entries in the mapping dictionary

In [12]:
correct = input('[V]iew, [A]dd, [R]emove or [U]pdate entry in the mapping_dict? [C]ancel ')
while correct.lower() != 'c':
# --------- view
  if correct.lower()=='v':
    view_entity = input('Enter entity to view ')
    if view_entity in mapping_dict:
      print(f"'{view_entity}' contains '{str(mapping_dict[view_entity])}'")
    else:
      print(f"'{view_entity}' not found!")
# --------- add
  elif correct.lower()=='a':
    add_entity = input('Enter entity to add ')
    if add_entity not in mapping_dict:
      new_ig_entity = input('Enter ig entity ')
      new_tag = input('Enter tag ')
      mapping_dict[add_entity] = {'ig':new_ig_entity, 'tag':{new_tag}}
      print(f"'{str(mapping_dict[add_entity])}' added successfully")
    else:
      print(f"'{add_entity}' already exists!")
# --------- remove
  elif correct.lower()=='r':
    remove_entity = input('Enter entity to remove ')
    if remove_entity in mapping_dict:
      removed = mapping_dict.pop(remove_entity)
      print(f"'{str(removed)}' successfully removed!")
    else:
      print(f"'{remove_entity}' not found!")
# --------- update
  elif correct.lower()=='u':
    update_entity = input('Enter entity to update ')
    if update_entity in mapping_dict:
      update_ig_entity = input(f"Update ig entity '{mapping_dict[update_entity]['ig']}'? [Y/N] ")
      if update_ig_entity.lower() == 'y':
        old_ig_entity = mapping_dict[update_entity]['ig']
        new_ig_entity = input('Enter new ig entity: ')
        mapping_dict[update_entity]['ig'] = new_ig_entity if new_ig_entity else old_ig_entity
        print(f" Changed ig entity: '{old_ig_entity}'==>'{new_ig_entity}'")
      update_tag = input(f"Update tag '{mapping_dict[update_entity]['tag']}'? [Y/N] ")
      if update_tag.lower() == 'y':
        old_tag = mapping_dict[update_entity]['tag']
        new_tag = input('Enter new tag: ')
        mapping_dict[update_entity]['tag'] = {new_tag} if new_tag else old_tag
        print(f"Changed ig entity: '{old_tag}'==>'{new_tag}'")
      print(f"'{str(mapping_dict[update_entity])}' updated successfully")
    else:
      print(f"'{update_entity}' not found!")
  else:
    print(f"Invalid input '{correct}'!")
  correct = input(f"{'-'*10}\n[V]iew, [A]dd, [R]emove or [U]pdate entry in the mapping_dict? [C]ancel ")
print("Thanks...")

KeyboardInterrupt: ignored

**OLD: Correcting the Mapping dictionary**

In [ ]:
# en_entity = input('Enter the dictionary key (english entity):')
# for entity, values in mapping_dict.items():
#   if values['ig'].startswith('Not found'):
#     index = int(values['ig'].rsplit(' ',1)[1])
#     print(f"Line {index}: {entity}\n-{tagged_entities[str(index)]}\n-{ig_text[index].lower()}")
#     ig_entity = input('Enter Igbo entity: ')
#     if ig_entity.lower() != 'exit':
#       mapping_dict[entity]['ig'] = ig_entity
#       print(f"Remaining: {count_not_found()}")
#     else:
#       break

(['Nigeria, LOC', 'Omoyele Sowore, PER'],
 'mụ onwe m nwere ebumnobi nye naịjirịa na gunye ndị amaala ọzọ dịka omoyele sowore ndị nke nwerela nkoropụ na mgbahapụ')

**Core Transfer Process** (Work in progress...)

In [1]:
ig_text = open('projection_ig_text.txt').readlines()

In [ ]:
with open('ig_transferred_ner_tags.txt', 'w', encoding='utf8') as ig_tagged_file:
  try:
    for lineNum in tagged_entities:
      entities_tags, ig_sent = tagged_entities[lineNum], ig_text[lineNum]
      for entity_tag in entities_tags:
        entity, tag = entity_tag.split(', ') #check this later
        ig_entity = mapping_dict[entity.lower()]['ig'].strip()
        tag = mapping_dict[entity.lower()]['tag']
        ent_start_index = ig_sent.find(ig_entity)
        # print(ig_entity, ent_start_index)
        if ent_start_index > -1:
          left_context = ig_sent[:ent_start_index]
          ig_tagged_file.write("\n".join([f"{tkn.strip()} O" for tkn in left_context.split()]))
          if ' ' not in ig_entity:
            ig_tagged_file.write(f"\n{ig_entity} B-{next(iter(tag))}")
          else:
            entity_tokens = ig_entity.split()
            entity_tok_tags = [f"{tkn.strip()} B-{next(iter(tag))}" if i==0 else 
                              f"{tkn.strip()} I-{next(iter(tag))}"
                              for i, tkn in enumerate(entity_tokens)]
            formated_tok_tags = '\n'.join(entity_tok_tags)
            ig_tagged_file.write(f"\n{formated_tok_tags}")
          right_context = ig_sent[ent_start_index+len(ig_entity):]
          ig_tagged_file.write(f"\n{'-'*3}\n")
          ig_sent = right_context
        else:
          print(f"Not found-> Line {lineNum} en: '{entity}' ig: '{ig_entity}'")
  except ValueError:
      print(f"ValueError: Line {lineNum}\n-entities_tags: '{entities_tags}'\n-ig_sent: '{ig_sent}'")

**Checking stuff**

In [ ]:
# TODO
mapping_dict